In [1]:
import numpy as np
import brian2 as br
import matplotlib.pyplot as plt
from time import time
from Reservoir.reservoir import Reservoir

In [2]:
res = Reservoir()
res.init_connection_matrix()

In [ ]:
neuron_group = br.NeuronGroup(res.n_neurons_side**3, eqs, threshold='v>1', reset='v=0')
S = br.Synapses(neuron_group, neuron_group, on_pre='')
S.connect(res.M)